In [1]:
from task_graph import TaskGraph, FUNCTION_REGISTRY
from function_group import FunctionGroup
import ndcctools.taskvine as vine
from utils import *
import dask
from dask.base import collections_to_dsk


class SimpleGroup:
    def __init__(self, keys, sexpr_of, input_paths, output_paths):
        self.keys = keys
        self.sexpr_of = sexpr_of

        # these keys include external parents
        self.input_paths = input_paths         # {key: input filename}
        self.output_paths = output_paths       # {key: output filename}
        
        # store results for keys in this group and its parents
        self.values = {}

class SimpleGroup:
    def __init__(self, keys, sexpr_of, input_paths, output_paths):
        self.keys = keys
        self.sexpr_of = sexpr_of

        # these keys include external parents
        self.input_paths = input_paths         # {key: input filename}
        self.output_paths = output_paths       # {key: output filename}
        
        # store results for keys in this group and its parents
        self.values = {}

def execute_converted_group(group: SimpleGroup):
    # load parent task outputs
    for k, path in group.input_paths.items():
        try:
            with open(path, 'rb') as f:
                group.values[k] = cloudpickle.load(f)
        except Exception as e:
            raise(f"Error loading input {k} from {path}: {e}")

    def rec_call(expr):
        try:
            if expr in group.values:
                return group.values[expr]
        except TypeError:
            pass
        if isinstance(expr, list):
            return [rec_call(e) for e in expr]
        if isinstance(expr, tuple) and isinstance(expr[0], str) and expr[0] in FUNCTION_REGISTRY:
            fn = FUNCTION_REGISTRY[expr[0]]
            return fn(*[rec_call(a) for a in expr[1:]])
        return expr

    for k in group.keys:
        result = rec_call(group.sexpr_of[k])
        group.values[k] = result

        if k in group.output_paths:
            with open(group.output_paths[k], 'wb') as f:
                cloudpickle.dump(result, f)

    return group.output_paths


class TaskGraphExecutor():
    def __init__(self, task_graph, lib_resources=None):
        self.manager = vine.Manager(
            9126,
            name="graph-optimization",
            # run_info_template="test2",
        )
        # self.manager.tune("watch-library-logfiles", 1)
        # self.manager.tune("temp-replica-count", 3)
        self.manager.tune("worker-source-max-transfers", 10000)
        self.manager.tune("max-retrievals", -1)
        self.libtask = self.manager.create_library_from_functions('library', execute_converted_group, add_env=False, hoisting_modules=[SimpleGroup, FUNCTION_REGISTRY])
        if lib_resources:
            if 'cores' in lib_resources:
                self.libtask.set_cores(lib_resources['cores'])
                self.libtask.set_function_slots(lib_resources['cores'])
            if 'memory' in lib_resources:
                self.libtask.set_memory(lib_resources['memory'])
            if 'disk' in lib_resources:
                self.libtask.set_disk(lib_resources['disk'])
            if 'slots' in lib_resources:
                self.libtask.set_function_slots(lib_resources['slots'])
        self.manager.install_library(self.libtask)

        self.task_graph = task_graph
        self.num_pending_parents_of_key = {k: v.in_degree() for k, v in self.task_graph.key_to_vertex.items()}
        self.ready_keys = deque([k for k, count in self.num_pending_parents_of_key.items() if count == 0])

    def create_group(self, key):
        group_keys = [key]
        group_sexpr_of = self.task_graph.get_sexpr_of_group(group_keys)

        group = SimpleGroup(
            keys=group_keys,
            sexpr_of=group_sexpr_of,
            input_paths={
                k: self.task_graph.output_filename_of[k]
                for k in self.task_graph.get_input_keys_of_group(group_keys)
            },
            output_paths={
                k: self.task_graph.output_filename_of[k]
                for k in self.task_graph.get_output_keys_of_group(group_keys)
            }
        )
        return group

    def execute(self):
        key_of_task = {}
        with create_progress_bar() as pbar:
            pbar_task = pbar.add_task("Executing task graph", total=len(self.task_graph.sexpr_of))
            while not pbar.tasks[pbar_task].finished:
                while self.ready_keys:
                    # submit all ready keys
                    rk = self.ready_keys.popleft()

                    # create a function call task
                    group = self.create_group(rk)
                    t = FunctionGroup('library', 'execute_converted_group', group)

                    # if rk not in self.task_graph.compute_keys:
                    t.enable_temp_output()
                    # else:
                    #     pass

                    t.set_cores(1)
                    t.set_priority(time.time())
                    key_of_task[t] = rk

                    # add input files
                    for k in self.task_graph.get_input_keys_of_group(group.keys):
                        t.add_input(self.task_graph.output_vine_file_of[k], self.task_graph.output_filename_of[k])

                    # add output files
                    # print(f"output keys: {self.task_graph.get_output_keys_of_group(group.keys)}")
                    for k in self.task_graph.get_output_keys_of_group(group.keys):
                    # TEMP HACK: use itself as the outkey
                    # for k in group.keys:
                        f = self.manager.declare_temp()
                        t.add_output(f, self.task_graph.output_filename_of[k])
                        self.task_graph.output_vine_file_of[k] = f

                    self.manager.submit(t)

                t = self.manager.wait(5)
                if t:
                    if t.successful():
                        task_key = key_of_task[t]
                        for child in self.task_graph.children_of[task_key]:
                            self.num_pending_parents_of_key[child] -= 1
                            if self.num_pending_parents_of_key[child] == 0:
                                self.ready_keys.append(child)
                        pbar.update(pbar_task, advance=1)
                    else:
                        print(f"failed")





In [6]:
tasks = load_from("tasks.pkl")
task_list = list(tasks.values())
dsk = collections_to_dsk(task_list)
keys = [t.key for t in task_list]

dsk
# task_graph = TaskGraph(dsk, keys, expand_dsk=True)

HighLevelGraph with 12 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x7f6b9e4ba350>
 0. with-field-c4c593bb3db0db4e51b57bb36cc9aa26
 1. btag-count-ec1b8d9ec2184716bb36bddd163c3d94
 2. ecfs-ab24aeaf4c441eddf3537cf574718cf5
 3. GenMatch-Mask-412de5427badce8c937311c708dc60b8
 4. goodjets-6238df45a5ee26211d6b72a19eec57f5
 5. color-ring-8e436b05b5d9168829b1d854943faf7f
 6. particleNet-HbbvsQCD-75ada7aaeca3f7367870bb2c85c5d2a2
 7. msoftdrop-4b19eba794f330f3a0a133d7ee033567
 8. particleNetMD-QCD-8f70636ab47869a500bdff54e917bd1b
 9. pt-3254da6395ba4a64ae620fe688067522
 10. to-parquet-990e1f103d04e6b01044065a0e1cc06b
 11. write-parquet-dace202fed69d93b22c79a2f0853a375-finalize

In [19]:
def flatten_hlg(hlg):
    assert isinstance(hlg, HighLevelGraph)
    task_dict = {}
    for k, sexpr in hlg.items():
        if isinstance(sexpr, SubgraphCallable):
            args = sexpr.args
            outkey = sexpr.outkey
            task_dict[k] = hash_name(k, outkey)
            for subkey, subexpr in sexpr.dsk.items():
                unique_key = hash_name(k, subkey)
                converted = TaskGraph.convert_expr_to_task_args(None, sexpr.dsk, subexpr, args)
                task_dict[unique_key] = converted
        elif isinstance(sexpr, tuple):
            task_dict[k] = sexpr
        elif isinstance(sexpr, (int, float, str)):
            task_dict[k] = sexpr
        else:
            raise ValueError(f"Unexpected type in HLG: {type(sexpr)}")
    return task_dict

from dask.optimization import SubgraphCallable

tasks = load_from("tasks.pkl")
task_list = list(tasks.values())
hlg = collections_to_dsk(task_list)
assert isinstance(hlg, HighLevelGraph)
task_dict = flatten_hlg(hlg)

In [31]:
import json
from graphviz import Digraph

def visualize_task_dict(json_path="task_dict.json", output_path="task_graph.svg"):
    with open(json_path, "r") as f:
        task_dict = json.load(f)

    keys = list(task_dict.keys())
    key_to_id = {k: str(i + 1) for i, k in enumerate(keys)}

    dot = Digraph(format="svg")
    dot.attr(dpi='300')

    # 添加节点
    for key, node_id in key_to_id.items():
        dot.node(node_id, label=node_id)

    # 递归提取依赖
    def extract_parents(expr):
        deps = set()
        if isinstance(expr, str) and expr in key_to_id:
            deps.add(expr)
        elif isinstance(expr, (list, tuple)):
            for e in expr:
                deps.update(extract_parents(e))
        return deps

    # 添加边
    for k, expr in task_dict.items():
        children = extract_parents(expr[1:] if isinstance(expr, list) else expr)
        for parent in children:
            dot.edge(key_to_id[parent], key_to_id[k])

    dot.render(output_path, view=False)
    print(f"✅ 任务图已保存为 {output_path}")


visualize_task_dict()

✅ 任务图已保存为 task_graph.svg


In [2]:
tasks = load_from("tasks.pkl")
task_list = list(tasks.values())
hlg = collections_to_dsk(task_list)
assert isinstance(hlg, HighLevelGraph)

def flatten_hlg(hlg):
    assert isinstance(hlg, HighLevelGraph)
    task_dict = {}
    for k, sexpr in hlg.items():
        if isinstance(sexpr, SubgraphCallable):
            args = sexpr.args
            outkey = sexpr.outkey
            task_dict[k] = hash_name(k, outkey)
            for subkey, subexpr in sexpr.dsk.items():
                unique_key = hash_name(k, subkey)
                converted = TaskGraph.convert_expr_to_task_args(None, sexpr.dsk, subexpr, args)
                task_dict[unique_key] = converted
        elif isinstance(sexpr, tuple):
            task_dict[k] = sexpr
        elif isinstance(sexpr, (int, float, str)):
            task_dict[k] = sexpr
        else:
            raise ValueError(f"Unexpected type in HLG: {type(sexpr)}")
    return task_dict

task_dict = flatten_hlg(hlg)

/users/jzhou24/miniconda3/envs/dv5/lib/python3.11/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [3]:
task_graph = TaskGraph(task_dict, expand_dsk=False, enable_sexpr_conversion=False)

In [4]:
task_graph